In [9]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

def init_spark():
  spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
  sc = spark.sparkContext
  return spark,sc

def main():
  spark,sc = init_spark()
  input_file = sc.textFile("input.txt")
  map = input_file.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
  counts = map.reduceByKey(lambda a, b: a + b)
  counts=counts.sortByKey()
  # filter out words with fewer than threshold occurrences
  filtered = counts.filter(lambda pair:pair[1] >= 1)

  # count characters
  charCounts = filtered.flatMap(lambda pair:pair[0]).map(lambda c: c).map(lambda c: (c, 1)).reduceByKey(lambda v1,v2:v1 +v2)

  list = charCounts.collect()
  print(repr(list)[1:-1])
  counts.saveAsTextFile("output3")


if __name__ == '__main__':
  main()

('b', 2), ('g', 3), ('i', 4), ('s', 3), ('y', 2), ('r', 2), ('l', 3), ('h', 3), ('d', 1), ('e', 6), ('v', 1), ('n', 4), ('m', 3), ('a', 3), ('x', 1), ('o', 3), ('u', 1), ('t', 2), ('w', 1)


In [27]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

def init_spark():
  spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
  sc = spark.sparkContext
  return spark,sc

def main():
  spark,sc = init_spark()
  rdd = sc.parallelize(transactions) \
        .map(lambda el: ((el['country'], el['name']),el['amount'])) \
        .partitionBy(4, country_partitioner)
  print("Number of partitions: {}".format(rdd.getNumPartitions()))
  print("Partitioner: {}".format(rdd.partitioner))
  print("Partitions structure: {}".format(rdd.glom().collect()))
  counts = rdd.reduceByKey(lambda a, b: str(a)+','+ str( b))
  counts=counts.sortByKey()
  print(counts.collect())
def country_partitioner(country):
    return hash(country)  



if __name__ == '__main__':
  main()

Number of partitions: 4
Partitioner: <pyspark.rdd.Partitioner object at 0x00000277FFD57898>
Partitions structure: [[], [(('United Kingdom', 'Bob'), 100), (('United Kingdom', 'Bob'), 15), (('Poland', 'Marek'), 51)], [(('Poland', 'Paul'), 75)], [(('Germany', 'Johannes'), 200)]]
[(('Germany', 'Johannes'), 200), (('Poland', 'Marek'), 51), (('Poland', 'Paul'), 75), (('United Kingdom', 'Bob'), '100,15')]


In [22]:
transactions = [
    {'name': 'Bob', 'amount': 100, 'country': 'United Kingdom'},
    {'name': 'Bob', 'amount': 15, 'country': 'United Kingdom'},
    {'name': 'Marek', 'amount': 51, 'country': 'Poland'},
    {'name': 'Johannes', 'amount': 200, 'country': 'Germany'},
    {'name': 'Paul', 'amount': 75, 'country': 'Poland'},
]

In [ ]:
import itertools
keyedElems = input.map(lambda x: (x[0], (x[1], x[2])))
groupedCombinations = keyedElems.groupByKey().flatMapValues(lambda arr: itertools.combinations(arr, 2))
productScoreCombinations = groupedCombinations.mapValues(lambda elems: ((elems[0][0], elems[1][0]), (elems[0][1], elems[1][1]))).map(lambda x: x[1])